In [2]:
import numpy as np
from tensorflow.keras.utils import plot_model, set_random_seed, to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, BatchNormalization, Conv2D, MaxPool2D, AveragePooling2D, ZeroPadding2D, add, Flatten
from tensorflow.keras.datasets import mnist

In [15]:
# load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [16]:
# from sparse label to categorical
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# reshape input images
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
y_train.shape

(60000, 10)

In [17]:
# This ensures that the model is deterministic
set_random_seed(seed=6)

def Conv2D_BN(x, nb_filter, kernel_size, strides=(1, 1), padding='same'):
    x = Conv2D(nb_filter, kernel_size, padding=padding, strides=strides, activation='relu')(x)
    x = BatchNormalization(axis=3)(x)
    return x

def Residual_Block(inpt, nb_filter, kernel_size, strides=(1, 1), with_conv_shortcut=False):
    x = Conv2D_BN(inpt, nb_filter=nb_filter, kernel_size=kernel_size, strides=strides, padding='same')
    x = Conv2D_BN(x, nb_filter=nb_filter, kernel_size=kernel_size, padding='same')
    if with_conv_shortcut:
        shortcut = Conv2D_BN(inpt, nb_filter=nb_filter, strides=strides, kernel_size=kernel_size)
        x = add([x, shortcut])
        return x
    else:
        x = add([x, inpt])
        return x

# Model definition
inpt = Input(shape=(28, 28, 1))
x = ZeroPadding2D(padding=(3, 3), data_format='channels_last')(inpt)
x = Conv2D_BN(x, nb_filter=64, kernel_size=(7, 7), strides=(2, 2), padding='valid')
x = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
# (56,56,64)
x = Residual_Block(x, nb_filter=64, kernel_size=(3, 3))
x = Residual_Block(x, nb_filter=64, kernel_size=(3, 3))
x = Residual_Block(x, nb_filter=64, kernel_size=(3, 3))
# (28,28,128)
x = Residual_Block(x, nb_filter=128, kernel_size=(3, 3), strides=(2, 2), with_conv_shortcut=True)
x = Residual_Block(x, nb_filter=128, kernel_size=(3, 3))
x = Residual_Block(x, nb_filter=128, kernel_size=(3, 3))
x = Residual_Block(x, nb_filter=128, kernel_size=(3, 3))
# (14,14,256)
x = Residual_Block(x, nb_filter=256, kernel_size=(3, 3), strides=(2, 2), with_conv_shortcut=True)
x = Residual_Block(x, nb_filter=256, kernel_size=(3, 3))
x = Residual_Block(x, nb_filter=256, kernel_size=(3, 3))
x = Residual_Block(x, nb_filter=256, kernel_size=(3, 3))
x = Residual_Block(x, nb_filter=256, kernel_size=(3, 3))
x = Residual_Block(x, nb_filter=256, kernel_size=(3, 3))
# (7,7,512)
x = Residual_Block(x, nb_filter=512, kernel_size=(3, 3), strides=(2, 2), with_conv_shortcut=True)
x = Residual_Block(x, nb_filter=512, kernel_size=(3, 3))
x = Residual_Block(x, nb_filter=512, kernel_size=(3, 3))
# x = AveragePooling2D(pool_size=(7, 7))(x)
x = AveragePooling2D(pool_size=(1, 1))(x)
x = Flatten()(x)
x = Dense(units=10, activation='softmax')(x)

# Create a Keras Model
model = Model(inputs=inpt, outputs=x)
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# train the model with input images and labels
model.fit(x=x_train,
          y=y_train,
          validation_data=(x_test, y_test),
          epochs=20,
          batch_size=32)

# model accuracy on test dataset
score = model.evaluate(x_test,
                       y_test,
                       batch_size=32,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))



Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 zero_padding2d_6 (ZeroPadding2  (None, 34, 34, 1)   0           ['input_7[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_216 (Conv2D)            (None, 14, 14, 64)   3200        ['zero_padding2d_6[0][0]']       
                                                                                                  
 batch_normalization_216 (Batch  (None, 14, 14, 64)  256         ['conv2d_216[0][0]']       

TensorShape([60000, 34, 34, 1])